In [1]:
'''
tcloud中用销售订单生成税局发票模板
1. 分别按每个销售订单汇总，再按存货编码汇总，批量生成发票模板，存于同一文件夹下
2. 分别按每个客户汇总，再按存货编码汇总，批量生成发票模板，存于同一文件夹下
3. 选择税率
全部自动生成


'''
import os
import re
import easygui
import openpyxl
import numpy as np
import pandas as pd
import shutil

In [2]:
def getPivot(df):
    bianba_fenlei = dict(zip(df['存货编码'],df['存货分类']))
    bianba_mingchen = dict(zip(df['存货编码'],df['存货名称']))
    bianba_daima = dict(zip(df['存货编码'],df['存货代码']))
    bianba_hanlian = dict(zip(df['存货编码'],df['件含量']))
    pivot = df.pivot_table(index = '存货编码',values = ['数量','数量（件）','含税金额'] ,aggfunc = 'sum')
    pivot = pivot.reset_index()
    pivot = pivot.assign(fenlei = pivot['存货编码'].map(bianba_fenlei))
    pivot = pivot.assign(mingchen = pivot['存货编码'].map(bianba_mingchen))
    pivot = pivot.assign(daima = pivot['存货编码'].map(bianba_daima))
    pivot = pivot.assign(hanliang = pivot['存货编码'].map(bianba_hanlian))
    dic = dict(zip(['fenlei','mingchen','daima','hanliang'],['存货分类','存货名称','存货代码','件含量']))
    pivot = pivot.rename(columns =  dic)
    guige_qian = []
    guige_hou = []
    for i in pivot['存货名称'].to_list():
        qian0,hou0 = guige(i)
        guige_qian.append(qian0)
        guige_hou.append(hou0)
    pivot = pivot.assign(qian = guige_qian)
    pivot = pivot.assign(hou = guige_hou)
    return pivot

def getFapiaoBen(d,shuilu,shuliang_fangsi):
    
    d['项目名称'] = d['存货编码'] +  d['hou']
    d['项目名称'] = d['项目名称'].str.replace('运费-','运费')
    # d['项目名称'] = d['项目名称'].str.split('-').str[0]
    d['商品和服务税收分类编码'] = '1060202010000000000'
    d['规格型号'] = d['qian']
    if shuliang_fangsi == '本数':
        d['单位'] = '本'
        d['商品数量'] = d['数量']
    else :
        d['单位'] = '件'
        d['商品数量'] = d['数量（件）']
        
    d['商品单价'] = ''
    d['金额'] = d['含税金额']
    d['税率'] = shuilu
    d['折扣金额'] = ''
    d['优惠政策类型'] = ''
    d = d[['项目名称',
     '商品和服务税收分类编码',
     '规格型号',
     '单位',
     '商品数量',
     '商品单价',
     '金额',
     '税率',
     '折扣金额',
     '优惠政策类型',]]
    return d

def getFapiaoMoban(gongsi,shuliang_fangsi):
    filename=''.join(['发票模板-',shuliang_fangsi,'-',gongsi,'.xlsx'])
    newname = os.path.join(path1,filename)
    shutil.copyfile(fname_fapiao, newname)
    return newname

def getFapiaoMobanDingdanhao(gongsi,shuliang_fangsi,dingdanhao):
    filename=''.join(['发票模板-',gongsi,'-',shuliang_fangsi,'-',dingdanhao,'.xlsx'])
    newname = os.path.join(path1,filename)
    shutil.copyfile(fname_fapiao, newname)
    return newname

def fengefu(string):
    num = len(string.split('-'))
    if num  == 3 :
        string = '-'.join([string.split('-')[0],string.split('-')[1]])
    elif num == 2 :
        if len(string.split('-')[0]) <= 4:
            string = string
        else :
            string = string.split('-')[0]
    else :
        string = string
    return string

def guige(string):
    if ('型' in string) and ('页' in string):
        qian0 = string.split('型')[0] + '型'
        hou0 =  string.split('型')[1] 
    elif ('型' in string) or ('页' in string):
        if '型' in string :
            qian0 = string.split('型')[0] + '型'
            hou0 =  string.split('型')[1] 
        else :
            qian0 = string.split('页')[0] + '页'
            hou0 =  string.split('页')[1] 
    else :
        qian0 = ''
        hou0 = ''
    return qian0,hou0    
            
    
    

In [3]:
path = r'F:\a00nutstore\008\zw08\莱新订单20241029'
os.chdir(path)
#莱新销售订单超5000条，不能一次导出，分三次导出，并分别存于同一文件下，先将它们合并
data = []
for i in os.listdir(path):
    if  os.path.isfile(i):
        i = os.path.join(path,i)
        df = pd.read_excel(i)
        data.append(df)
    else :
        continue
   

df_xiaoshou0 = pd.concat(data)
df_xiaoshou1 = df_xiaoshou0.loc[df_xiaoshou0['单据执行状态'] != '合计']
lst1 = ['单据编号',
  '单据日期',
 '含税总金额',
 '存货名称',
 '存货分类',
 '存货编码',
  '存货代码',
  '数量',
  '件含量',
 '数量（件）',
  '含税单价',
 '含税金额',
        '客户']
df_xiaoshou2 = df_xiaoshou1[lst1]
df_xiaoshou3 = df_xiaoshou2.copy()
s = []
for i in df_xiaoshou2['存货编码'].to_list():
    j = fengefu(i)
    s.append(j)
df_xiaoshou3['存货编码'] = s

In [4]:
#选择税率
shuilu = easygui.choicebox(msg = '请选择税率',choices = [0.13,0.01,0.02])
shuilu  = float(shuilu)
#选择数量的开具方式，件数or本数
shuliang_fangsi = easygui.choicebox(msg = '请选择数量开具方式',choices = ['件数','本数'])
# #选择发票开具方式
# fapiao_fangsi = easygui.choicebox(msg = '请选择发票开具方式',choices = ['按客户和存货编码','按销售订单和存货编码','按客户汇总'])

In [5]:
#读取发票模板
fname_fapiao = r"F:\repos\fish\发票模板.xlsx"
df_fapiao = pd.read_excel(fname_fapiao,header = 2,dtype = {'商品和服务税收分类编码':'str'})
df_fapiao

,项目名称,商品和服务税收分类编码,规格型号,单位,商品数量,商品单价,金额,税率,折扣金额,优惠政策类型


In [6]:
#按照发票开具方式，及数量选择方式，生成对应的文件夹
path1 = f'发票模板-按客户和存货编码-{shuliang_fangsi}'
path1 = os.path.join(path,path1)
if not os.path.exists(path1):
    try:
        os.mkdir(path1)
    except:
        pass

gp = df_xiaoshou3.groupby('客户')
for k,v  in gp:
    gongsi = k
    newname = getFapiaoMoban(gongsi,shuliang_fangsi)
    pivot = getPivot(v)
    # pivot1 = chuliMingchen(pivot)
    fapiao = getFapiaoBen(pivot,shuilu,shuliang_fangsi)
    with pd.ExcelWriter(newname, engine='openpyxl',mode='a', if_sheet_exists='overlay')  as writer:
        fapiao.to_excel(writer, sheet_name = '1-明细模板',startrow=3, header = False,index = False)

#按照发票开具方式，及数量选择方式，生成对应的文件夹
path1 = f'发票模板-按销售订单和存货编码-{shuliang_fangsi}'
path1 = os.path.join(path,path1)
if not os.path.exists(path1):
    try:
        os.mkdir(path1)
    except:
        pass

gp = df_xiaoshou3.groupby('单据编号')
for k,v  in gp:
    dingdanhao = k
    gongsi = v['客户'].to_list()[0]
    newname = getFapiaoMobanDingdanhao(gongsi,dingdanhao,shuliang_fangsi)
    print(newname)
    pivot = getPivot(v)
    # pivot1 = chuliMingchen(pivot)
    fapiao = getFapiaoBen(pivot,shuilu,shuliang_fangsi)
    with pd.ExcelWriter(newname, engine='openpyxl',mode='a', if_sheet_exists='overlay')  as writer:
        fapiao.to_excel(writer, sheet_name = '1-明细模板',startrow=3, header = False,index = False)

#按照发票开具方式，及数量选择方式，生成对应的文件夹
path1 = f'发票模板-按客户汇总-{shuliang_fangsi}'
path1 = os.path.join(path,path1)
if not os.path.exists(path1):
    try:
        os.mkdir(path1)
    except:
        pass
     
gp = df_xiaoshou3.groupby('客户')
for k,v  in gp:
    gongsi = k
    newname = getFapiaoMoban(gongsi,shuliang_fangsi)
    v1 = v.sum().T.to_frame().T
    v1.loc[0,'存货名称'] = '*印刷品*本册'
    v1.loc[0,'存货分类'] = '*印刷品*本册'
    v1.loc[0,'存货编码'] = '*印刷品*本册'
    v1.loc[0,'存货代码'] = '*印刷品*本册'
    pivot = getPivot(v1)
    # pivot1 = chuliMingchen(pivot)
    fapiao = getFapiaoBen(pivot,shuilu,shuliang_fangsi)
    with pd.ExcelWriter(newname, engine='openpyxl',mode='a', if_sheet_exists='overlay')  as writer:
        fapiao.to_excel(writer, sheet_name = '1-明细模板',startrow=3, header = False,index = False)


    


C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is no

F:\a00nutstore\008\zw08\莱新订单20241029\发票模板-按销售订单和存货编码-件数\发票模板-河南抬头喜办公用品有限公司S-SO-2024-02-1182-件数.xlsx


C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


F:\a00nutstore\008\zw08\莱新订单20241029\发票模板-按销售订单和存货编码-件数\发票模板-武汉开先文化用品有限责任公司-SO-2024-02-1184-件数.xlsx


C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


...

C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


F:\a00nutstore\008\zw08\莱新订单20241029\发票模板-按销售订单和存货编码-件数\发票模板-武汉米圣贸易有限公司-SO-2024-02-1574-件数.xlsx


C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


F:\a00nutstore\008\zw08\莱新订单20241029\发票模板-按销售订单和存货编码-件数\发票模板-河南抬头喜办公用品有限公司S-SO-2024-02-1575-件数.xlsx


C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is no

In [7]:

    
    
    
    
    
    pivot
    
    
    
    
    


,存货编码,含税金额,数量,数量（件）,存货分类,存货名称,存货代码,件含量,qian,hou,项目名称,商品和服务税收分类编码,规格型号,单位,商品数量,商品单价,金额,税率,折扣金额,优惠政策类型
0,*印刷品*本册,6791.45,4892,23.0,*印刷品*本册,*印刷品*本册,*印刷品*本册,4292.0,,,*印刷品*本册,1060202010000000000,,件,23.0,,6791.45,0.13,,


In [8]:
gp

In [9]:
df00 = pd.DataFrame(gp)
df00

,0,1
0,上海流金岁月商贸有限公司,单据编号 单据日期 含税总金额 ...
1,临沂博涵文化用品有限公司,单据编号 单据日期 含税总金额 ...
2,临沂巧纳金纸品有限公司,单据编号 单据日期 含税总金额 ...
3,临沂李加朋,单据编号 单据日期 含税总金额 ...
4,义乌市博迅文体用品有限公司,单据编号 单据日期 含税总金额 ...
...,...,...
99,阜阳俊武文体商贸有限公司,单据编号 单据日期 含税总金额 ...
100,陕西富明文体用品有限公司,单据编号 单据日期 含税总金额 ...
101,青海一诺商贸有限公司,单据编号 单据日期 含税总金额 ...
102,鹤岗市超东选商贸有限公司,单据编号 单据日期 含税总金额 ...


In [10]:
df_xiaoshou3

,单据编号,单据日期,含税总金额,存货名称,存货分类,存货编码,存货代码,数量,件含量,数量（件）,含税单价,含税金额,客户
0,SO-2024-02-1418,2024-10-16,14064.05,A540型卡面无线胶订本,无线胶装软抄,N0671,抄本,3200,320.0,10.0,0.8352,2672.64,海南新宇晨办公用品有限公司
1,SO-2024-02-1418,2024-10-16,14064.05,A560型卡面无线胶订本,无线胶装软抄,N0672,抄本,1440,240.0,6.0,1.1712,1686.53,海南新宇晨办公用品有限公司
2,SO-2024-02-1418,2024-10-16,14064.05,A580型卡面无线胶订本,无线胶装软抄,N0673,抄本,540,180.0,3.0,1.5456,834.62,海南新宇晨办公用品有限公司
3,SO-2024-02-1418,2024-10-16,14064.05,A5100型卡面无线胶订本,无线胶装软抄,N0674,抄本,560,140.0,4.0,1.8912,1059.07,海南新宇晨办公用品有限公司
4,SO-2024-02-1418,2024-10-16,14064.05,A540型牛卡无线胶订本,无线胶装软抄,N0681,抄本,1600,320.0,5.0,0.8352,1336.32,海南新宇晨办公用品有限公司
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,SO-2024-02-1419,2024-10-15,47469.60,16K银行存款日记账,纸面账簿,6003,账本,300,60.0,5.0,5.5440,1663.20,品壹佰河北商贸有限公司
2160,SO-2024-02-1419,2024-10-15,47469.60,A480型牛皮无线胶订本,无线胶装软抄,N0697,抄本,100,100.0,1.0,3.3840,338.40,品壹佰河北商贸有限公司
2161,SO-2024-02-1419,2024-10-15,47469.60,A4100型牛皮无线胶订本,无线胶装软抄,N0698,抄本,80,80.0,1.0,4.3710,349.68,品壹佰河北商贸有限公司
2162,SO-2024-02-1419,2024-10-15,47469.60,16K现金日记账,纸面账簿,6105,账本,200,40.0,5.0,7.8720,1574.40,品壹佰河北商贸有限公司


In [12]:
df_xiaoshou3.to_excel('beforr_gp.xlsx')

In [51]:
dff = df_xiaoshou3.copy()

In [14]:
dff['存货编码'].str.isinstance('int')

AttributeError: 'StringMethods' object has no attribute 'isinstance'

In [15]:
'111'.isinstance('int')

AttributeError: 'str' object has no attribute 'isinstance'

In [17]:
isinstance(111,int)

True

In [23]:
def chuliZhangben(str1,str2):
    if isinstance(str1,int):
        if 'K' in str2:
            qian,hou = str2.split('K')
        elif 'k' in str2:
            qian,hou = str2.split('k')
        else :
            qian = ''
            hou = str2
            
        str1 = str(str1) + 'hou'
        str2 = str2
    else:
        str1 = str1
        str2 = str2
    return str1,str2
    

In [29]:
chuliZhangben(111,'48K菜单')

('111hou', '48K菜单')

In [28]:
dff = dff.assign(kaibie=dff.map(lambda x,y: chuliZhangben(dff['存货编码'],dff['存货名称’])
                                                          
                                                          
                                                          

SyntaxError: unterminated string literal (detected at line 1) (132992160.py, line 1)

In [25]:
dff = dff.assign(kaibie=dff.map(lambda x,y: chuliZhangben(dff['存货编码'],dff['存货名称’]))))
                                                          

SyntaxError: invalid character '］' (U+FF3D) (2288988702.py, line 1)

ValueError: invalid literal for int() with base 10: 'N0671'

In [42]:
def chuliZhangben(str1):
    try :
        str1 = int(str1)
        if isinstance(str1,int):
            return True
        else:
            return False
    except:
        str1 = str1
        return False

        
          
   

In [43]:
dff['存货编码'].map(lambda x:chuliZhangben(x))

0       False
1       False
2       False
3       False
4       False
        ...  
2159     True
2160    False
2161    False
2162     True
2163     True
Name: 存货编码, Length: 3326, dtype: bool

In [65]:
dff1 = dff.assign(qian1 = lambda x:x['存货名称'].str.split('K')[0])
dff1

ValueError: cannot reindex on an axis with duplicate labels

In [59]:
dff['存货名称']

0        A540型卡面无线胶订本
1        A560型卡面无线胶订本
2        A580型卡面无线胶订本
3       A5100型卡面无线胶订本
4        A540型牛卡无线胶订本
            ...      
2159       16K银行存款日记账
2160     A480型牛皮无线胶订本
2161    A4100型牛皮无线胶订本
2162         16K现金日记账
2163               运费
Name: 存货名称, Length: 3326, dtype: object

In [61]:
qian1

0        [A540型卡面无线胶订本]
1        [A560型卡面无线胶订本]
2        [A580型卡面无线胶订本]
3       [A5100型卡面无线胶订本]
4        [A540型牛卡无线胶订本]
             ...       
2159      [16, 银行存款日记账]
2160     [A480型牛皮无线胶订本]
2161    [A4100型牛皮无线胶订本]
2162        [16, 现金日记账]
2163               [运费]
Name: 存货名称, Length: 3326, dtype: object

In [54]:
dff2 = dff.assign(he = np.where(dff['存货编码'].map(lambda x:chuliZhangben(x)),'X','Y'))

ValueError: operands could not be broadcast together with shapes (3326,) (2,) () 

In [53]:
dff1.to_excel('dff.xlsx')